# Setting up the environment

We'll load the needed libraries:


In [3]:
options(repr.matrix.max.rows=100, repr.matrix.max.cols=300)
options(repr.plot.width = 20, repr.plot.height = 15)
options(width=300)

numcores=70

library(tidyverse)
library(data.table)
library(fst)
library(comorbidity)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
`%!in%` = Negate(`%in%`)

setDTthreads(numcores)

# Codes

First, we will add codes from ICD and Medicare:primary_care_specialty_codes

In [8]:
#diagnosis codes

office_visit_codes=c("99201","99202","99203","99204","99205","99211","99212","99213","99214"
                     ,"99215")

IHD_icd_9_codes=c(410, 411, 412,413,414)
IHD_icd_10_codes=c("I20", "I21", "I22", "I23", "I24", "I25")

non_us_state_codes=c(40,54,56,57,58,59,60,61,62,63,64,65,66,97,98,99)

primary_care_specialty_codes=c("01", "08", "11", "38")

#http://www.icd9data.com/2015/Volume1/390-459/401-405/default.htm
#https://www.icd10data.com/ICD10CM/Codes/I00-I99/I10-I16
hypertension_icd_9_codes=c("401","402","403","404","405")
hypertension_icd_10_codes=c("I10","I11","I12","I13","I15","I16")

#http://www.icd9data.com/2014/Volume1/290-319/295-299/296/default.htm
#https://www.icd10data.com/ICD10CM/Codes/F01-F99/F30-F39
depression_icd_9_codes=c("2962","2963")
depression_icd_10_codes=c("F32","F33")

#http://www.icd9data.com/2015/Volume1/240-279/249-259/default.htm
#https://www.icd10data.com/ICD10CM/Codes/E00-E89/E08-E13
diabetes_icd_9_codes=c("250")
diabetes_icd_10_codes=c("E08","E09","E10","E11","E13")

#http://www.icd9data.com/2014/Volume1/710-739/710-719/714/default.htm
#https://www.icd10data.com/ICD10CM/Codes/M00-M99/M05-M14
arthritis_icd_9_codes=c("714")
arthritis_icd_10_codes=c("M05","M06","M07","M08","M09","M10","M11","M12","M13","M14")


race_codes=data.frame(race_code = seq(0,6),
                      race = c("Unknown", "White", "Black", "Other", "Asian", "Hispanic", "North American Native")
                      )

sex_codes=data.frame(sex_code = seq(0,2),
                    sex = c("Unknown","Male","Female"))


# Patient level calculations

## Yearly Calculators

These are the main functions that calculate yearly expenditures for patients and their corresponding physicians.\

### Fixing the MBSF data

In [10]:
mbsf_2013=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2013.csv" , num_threads = numcores) %>% as.data.table
mbsf_2014=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2014.csv" , num_threads = numcores) %>% as.data.table
mbsf_2015=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2015.csv" , num_threads = numcores) %>% as.data.table
mbsf_2016=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2016.csv" , num_threads = numcores) %>% as.data.table
mbsf_2017=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2017.csv" , num_threads = numcores) %>% as.data.table
mbsf_2018=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2018.csv" , num_threads = numcores) %>% as.data.table
mbsf_2019=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2019.csv" , num_threads = numcores) %>% as.data.table
mbsf_2020=vroom(file="/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2020.csv" , num_threads = numcores) %>% as.data.table

New names:
• `1` -> `1...5`
• `1` -> `1...7`
• `0` -> `0...8`
• `0` -> `0...9`
• `0` -> `0...11`
• `0` -> `0...12`
• `3` -> `3...13`
• `3` -> `3...14`
• `3` -> `3...15`
• `3` -> `3...16`
• `3` -> `3...17`
• `3` -> `3...18`
• `3` -> `3...19`
• `3` -> `3...20`
• `3` -> `3...21`
• `3` -> `3...22`
• `3` -> `3...23`
• `3` -> `3...24`
• `C` -> `C...25`
• `C` -> `C...26`
• `C` -> `C...27`
• `C` -> `C...28`
• `C` -> `C...29`
• `C` -> `C...30`
• `C` -> `C...31`
• `C` -> `C...32`
• `C` -> `C...33`
• `C` -> `C...34`
• `C` -> `C...35`
• `C` -> `C...36`
• `12` -> `12...37`
• `12` -> `12...38`
• `12` -> `12...39`
• `0` -> `0...40`
• `` -> `...41`
• `` -> `...42`
Rows: 2758309 Columns: 43
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (29): 000000000, 05, 400, 0...9, 3...13, 3...14, 3...15, 3...16, 3...17, 3...18, 3...19, 3...20, 3...21, 3...22, 3...23, 3...24, C...25, C...26, C...27, C...

In [11]:
mbsf_colnames_2013_2015=c("DESY_SORT_KEY","STATE_CODE","COUNTY_CODE","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MEDICARE_STATUS_CD","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","REFERENCE_YEAR")

mbsf_colnames_2016_2020=c("DESY_SORT_KEY","REFERENCE_YEAR","SAMPLE_GROUP","STATE_CODE","COUNTY_CODE","STATE_CNTY_FIPS_CD_01","STATE_CNTY_FIPS_CD_02","STATE_CNTY_FIPS_CD_03","STATE_CNTY_FIPS_CD_04","STATE_CNTY_FIPS_CD_05","STATE_CNTY_FIPS_CD_06","STATE_CNTY_FIPS_CD_07","STATE_CNTY_FIPS_CD_08","STATE_CNTY_FIPS_CD_09","STATE_CNTY_FIPS_CD_10","STATE_CNTY_FIPS_CD_11","STATE_CNTY_FIPS_CD_12","SEX_CODE","RACE_CODE","AGE","ORIG_REASON_FOR_ENTITLEMENT","CURR_REASON_FOR_ENTITLEMENT","ESRD_INDICATOR","MDCR_STATUS_CODE_01","MDCR_STATUS_CODE_02","MDCR_STATUS_CODE_03","MDCR_STATUS_CODE_04","MDCR_STATUS_CODE_05","MDCR_STATUS_CODE_06","MDCR_STATUS_CODE_07","MDCR_STATUS_CODE_08","MDCR_STATUS_CODE_09","MDCR_STATUS_CODE_10","MDCR_STATUS_CODE_11","MDCR_STATUS_CODE_12","PART_A_TERMINATION_CODE","PART_B_TERMINATION_CODE","ENTITLEMENT_BUY_IN_IND01","ENTITLEMENT_BUY_IN_IND02","ENTITLEMENT_BUY_IN_IND03","ENTITLEMENT_BUY_IN_IND04","ENTITLEMENT_BUY_IN_IND05","ENTITLEMENT_BUY_IN_IND06","ENTITLEMENT_BUY_IN_IND07","ENTITLEMENT_BUY_IN_IND08","ENTITLEMENT_BUY_IN_IND09","ENTITLEMENT_BUY_IN_IND10","ENTITLEMENT_BUY_IN_IND11","ENTITLEMENT_BUY_IN_IND12","HMO_INDICATOR01","HMO_INDICATOR02","HMO_INDICATOR03","HMO_INDICATOR04","HMO_INDICATOR05","HMO_INDICATOR06","HMO_INDICATOR07","HMO_INDICATOR08","HMO_INDICATOR09","HMO_INDICATOR10","HMO_INDICATOR11","HMO_INDICATOR12","HI_COVERAGE","SMI_COVERAGE","HMO_COVERAGE","STATE_BUY_IN_COVERAGE","VALID_DATE_OF_DEATH_SWITCH","DATE_OF_DEATH","DUAL_STUS_CD_01","DUAL_STUS_CD_02","DUAL_STUS_CD_03","DUAL_STUS_CD_04","DUAL_STUS_CD_05","DUAL_STUS_CD_06","DUAL_STUS_CD_07","DUAL_STUS_CD_08","DUAL_STUS_CD_09","DUAL_STUS_CD_10","DUAL_STUS_CD_11","DUAL_STUS_CD_12")


colnames(mbsf_2013)=mbsf_colnames_2013_2015
colnames(mbsf_2014)=mbsf_colnames_2013_2015
colnames(mbsf_2015)=mbsf_colnames_2013_2015
colnames(mbsf_2016)=mbsf_colnames_2016_2020
colnames(mbsf_2017)=mbsf_colnames_2016_2020
colnames(mbsf_2018)=mbsf_colnames_2016_2020
colnames(mbsf_2019)=mbsf_colnames_2016_2020
colnames(mbsf_2020)=mbsf_colnames_2016_2020

In [35]:
mbsf_needed_cols = c(
    "DESY_SORT_KEY"
    ,"REFERENCE_YEAR"
    ,"STATE_CODE"
    ,"COUNTY_CODE"
    ,"SEX_CODE"
    ,"RACE_CODE"
    ,"AGE"
    ,"ORIG_REASON_FOR_ENTITLEMENT"
    ,"CURR_REASON_FOR_ENTITLEMENT"
    ,"ENTITLEMENT_BUY_IN_IND01"
    ,"ENTITLEMENT_BUY_IN_IND02"
    ,"ENTITLEMENT_BUY_IN_IND03"
    ,"ENTITLEMENT_BUY_IN_IND04"
    ,"ENTITLEMENT_BUY_IN_IND05"
    ,"ENTITLEMENT_BUY_IN_IND06"
    ,"ENTITLEMENT_BUY_IN_IND07"
    ,"ENTITLEMENT_BUY_IN_IND08"
    ,"ENTITLEMENT_BUY_IN_IND09"
    ,"ENTITLEMENT_BUY_IN_IND10"
    ,"ENTITLEMENT_BUY_IN_IND11"
    ,"ENTITLEMENT_BUY_IN_IND12"
    ,"HMO_INDICATOR01"
    ,"HMO_INDICATOR02"
    ,"HMO_INDICATOR03"
    ,"HMO_INDICATOR04"
    ,"HMO_INDICATOR05"
    ,"HMO_INDICATOR06"
    ,"HMO_INDICATOR07"
    ,"HMO_INDICATOR08"
    ,"HMO_INDICATOR09"
    ,"HMO_INDICATOR10"
    ,"HMO_INDICATOR11"
    ,"HMO_INDICATOR12"
    ,"VALID_DATE_OF_DEATH_SWITCH"
    ,"DATE_OF_DEATH")

mbsf_data=list(mbsf_2013,mbsf_2014,mbsf_2015,mbsf_2016,mbsf_2017,mbsf_2018,mbsf_2019,mbsf_2020)

for (a in 1:length(mbsf_data)){
  mbsf_data[[a]]=mbsf_data[[a]][, ..mbsf_needed_cols]
}

#rename last n columns in a dataset
rename_last = function(data, how_many, new_names) {
  total_cols = ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}


for (a in 1:length(mbsf_data)){
  mbsf_data[[a]][,year:= 2012+a]
}



mbsf_data = reduce(mbsf_data, function(x,y) {rbind(x,y)}) %>% as.data.table()
                   


In [37]:
head(mbsf_data)

DESY_SORT_KEY,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,year
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
000000000,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,33,420,2,1,93,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,49,801,2,1,71,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,33,400,2,1,75,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,10,510,1,1,70,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013


In [38]:
mbsf_data_death_collapsed = mbsf_data[!is.na(DATE_OF_DEATH)]

mbsf_data_death_collapsed$date_died=mbsf_data_death_collapsed$DATE_OF_DEATH

mbsf_data = 
left_join(mbsf_data,
          mbsf_data_death_collapsed[,.(DESY_SORT_KEY,
                                       date_died)],
          by=("DESY_SORT_KEY")
         ) %>% as.data.table

mbsf_data_death_collapsed = mbsf_data[!is.na(VALID_DATE_OF_DEATH_SWITCH)]

mbsf_data_death_collapsed$date_died_valid=mbsf_data_death_collapsed$VALID_DATE_OF_DEATH_SWITCH

mbsf_data = 
left_join(mbsf_data,
          mbsf_data_death_collapsed[,.(DESY_SORT_KEY,
                                       date_died_valid)],
          by=("DESY_SORT_KEY")
         ) %>% as.data.table


head(mbsf_data)

DESY_SORT_KEY,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,year,date_died
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
000000000,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017
000000000,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627
000000000,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017
000000000,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627
000000000,13,33,420,2,1,93,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017
000000000,13,33,420,2,1,93,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627


In [43]:
write_fst(mbsf_data, "/work/postresearch/Shared/Projects/Farbod/CaseMix/mbsf_data_long.fst")

### Read data



In [ ]:
carrier_data_all_years = read_fst(
    "/work/postresearch/Shared/Projects/Farbod/carrier_data_all_years.fst", as.data.table = T)
mbsf_data = read_fst(
  "/work/postresearch/Shared/Projects/Farbod/CaseMix/mbsf_data_long.fst", as.data.table = T)
mbsf_data[,DESY_SORT_KEY := as.integer(DESY_SORT_KEY)]

In [ ]:
head(carrier_data_all_years)
head(mbsf_data)

### Loading sample data (for pc)

In [4]:
sample_data=readRDS(file = "sample_data.RDS")

In [5]:
carrier_data_all_years=sample_data[[1]]
outpatient_data_all_years=sample_data[[2]]
inpatient_data_all_years=sample_data[[3]]
mbsf_data=read_fst("mbsf_data_long.fst",as.data.table=T)
revenue_center_outpatient_all_years=sample_data[[5]]
outpatient_and_revenue_center_data=sample_data[[6]]

### Patient yearly expenditures and use of services carrier

I will first create a function that adds conditions of interest to the data.


#### Finding conditions for each claim line

In [46]:
yearly_calculator_patient_conditions = function(data) {
  
  #requirements
  require(data.table)
  require(dtplyr)
  require(tidyverse)
  require(lubridate)
  
  data %>%
    mutate(
      is_office_visit = HCPCS_CD %in% office_visit_codes,
      
      is_by_primary_care_physician= PRVDR_SPCLTY %in% primary_care_specialty_codes,

      is_hypertension= if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% hypertension_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% hypertension_icd_9_codes,NA)),
      
      is_arthritis= if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% arthritis_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% arthritis_icd_9_codes,NA)),
      
      is_IHD = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% IHD_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% IHD_icd_9_codes,NA)),
      
      is_diabetes= if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% diabetes_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% diabetes_icd_9_codes,NA)),
      
      is_depression= if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% depression_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 4) %in% depression_icd_9_codes,NA))

      
    ) %>%
    as.data.table()
}

yearly_patient_conditions_carrier=yearly_calculator_patient_conditions(carrier_data_all_years)
head(yearly_patient_conditions_carrier)


DESY_SORT_KEY,CLAIM_NO,LINE_NUM,CLM_THRU_DT,LINE_PLACE_OF_SRVC_CD,HCPCS_CD,LINE_ICD_DGNS_VRSN_CD,LINE_ICD_DGNS_CD,LINE_ALOWD_CHRG_AMT,PRF_PHYSN_NPI,PRVDR_SPCLTY,PRVDR_STATE_CD,date,year,month_year,is_office_visit,is_by_primary_care_physician,is_hypertension,is_arthritis,is_IHD,is_diabetes,is_depression
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<date>,<dbl>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
100000015,2,1,20130425,22,94375,9,496,15.26,1073503884,29,22,2013-04-25,2013,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2,2,20130425,22,94726,9,496,13.54,1073503884,29,22,2013-04-25,2013,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2,3,20130425,22,94729,9,496,9.95,1073503884,29,22,2013-04-25,2013,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,3,1,20130528,11,99214,9,41400,114.64,1285600932,11,22,2013-05-28,2013,2013-05,TRUE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE
100000015,3,2,20130528,11,93000,9,41400,20.08,1285600932,11,22,2013-05-28,2013,2013-05,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE
100000015,4,1,20130719,22,99213,9,496,51.76,1659344091,29,22,2013-07-19,2013,2013-07,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


#### Summarizing patient data
I will now summarise the data for each patient.


In [ ]:
summarise_carrier = function(data, time_frame = 365){
  
  data%>%
    group_by(DESY_SORT_KEY,year) %>%
    summarise(
      #tot_allowed_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT),
      
      #office_visit_count = sum(na.rm = T, is_office_visit),
      
      #office_visit_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_office_visit),
      
      distinct_clinicians = length(unique(PRF_PHYSN_NPI)),
      
      distinct_primary_care_physicians = length(.[is_by_primary_care_physician, unique(PRF_PHYSN_NPI)]),

      hypertension = sum(is_hypertension, na.rm = T) > 0,
      
      arthritis = sum(is_arthritis, na.rm = T) > 0,
      
      IHD = sum(is_IHD, na.rm = T) > 0,
      
      diabetes = sum(is_diabetes, na.rm = T) > 0,
  
      depression = sum(is_depression, na.rm = T) > 0,
      
      icd_9_pure = ifelse(prod(LINE_ICD_DGNS_VRSN_CD, na.rm = T) == 0, F, T),
      
      icd_10_pure = ifelse(sum(LINE_ICD_DGNS_VRSN_CD, na.rm = T) == 0, T, F),
      
    ) %>%
    as.data.table()
}


summary_patient_by_year = summarise_carrier(yearly_patient_conditions_carrier)
head(summary_patient_by_year)


`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.


In [ ]:
add_patient_characteristics = function(mbsf_data,summary_data){
  require(dtplyr)
  require(lubridate)
  require(tidyverse)
  data = left_join(summary_data,mbsf_data,by=c("DESY_SORT_KEY","year")) %>% as.data.frame()
  
  data %>%
  mutate(
    year_of_death=substr(date_died,0,4)
  )%>%
  as.data.table()
}

summary_with_patient_characteristics=add_patient_characteristics(mbsf_data,summary_patient_by_year)
head(summary_with_patient_characteristics)

In [ ]:
write.fst(summary_with_patient_characteristics,"summary_with_patient_characteristics_before_join.fst")
write.fst(summary_patient_by_year,"summary_patient_by_year_before_join.fst")


### Most common physicians for each patient

Now, we will find most common physicians and cardiologists for each patient.


In [ ]:
summary_with_patient_characteristics=read.fst("summary_with_patient_characteristics_before_join.fst", as.data.table=T)
summary_patient_by_year=read.fst("summary_patient_by_year_before_join.fst", as.data.table=T)


In [ ]:
#adding most common physicians
add_patient_NPI=function(data, summary_data, time_frame = 365){

  comorbidity_and_phys_data =
    inner_join(data, summary_data[, c("DESY_SORT_KEY","year",
                                "icd_9_pure",
                                "icd_10_pure")], by = c("DESY_SORT_KEY","year")) %>%
    as.data.table()
  
  patient_NPI_count_finder = function(data) {
    result = data %>%
      mutate(is_office_visit = HCPCS_CD %in% office_visit_codes)%>%
      group_by(DESY_SORT_KEY, year, PRF_PHYSN_NPI) %>%
      summarise(n = sum(is_office_visit,na.rm=T)) %>%
      filter(n>0)%>%
      arrange(.by_group = T, desc(n))
  }
  
  patient_NPI_counts = patient_NPI_count_finder(comorbidity_and_phys_data)
  
  patient_NPI_counts = left_join(patient_NPI_counts,
                                 distinct(data[, .(PRF_PHYSN_NPI, PRVDR_SPCLTY)]), by ="PRF_PHYSN_NPI")
  
  find_most_common = function(data) {
    data %>%
      group_by(DESY_SORT_KEY,year) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }
  
  find_most_common_by_specialty = function(data, specialty_code) {
    data %>%
      filter(PRVDR_SPCLTY %in% specialty_code) %>%
      group_by(DESY_SORT_KEY,year) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }
  
  most_common_physician = find_most_common(patient_NPI_counts)
  most_common_primary_care_physician = find_most_common_by_specialty(patient_NPI_counts,
                                                                     specialty_code = c("01", "08", "11", "38"))
  most_common_physician = data.frame(most_common_physician) %>%
    rename_with( ~ paste0("most_common_physician_", .x))
  most_common_primary_care_physician = data.frame(most_common_primary_care_physician) %>%
    rename_with( ~ paste0("most_common_primary_care_physician_", .x))
  
  summary_data = left_join(
    summary_data,
    most_common_physician,
    by = c("DESY_SORT_KEY" = "most_common_physician_DESY_SORT_KEY" , "year" = "most_common_physician_year")
  )
  summary_data = left_join(
    summary_data,
    most_common_primary_care_physician,
    by = c("DESY_SORT_KEY" = "most_common_primary_care_physician_DESY_SORT_KEY" , "year" = "most_common_primary_care_physician_year")
  )%>%
  as.data.table()

}

summary_with_npi=add_patient_NPI(data = carrier_data_all_years, summary_data = summary_with_patient_characteristics)
head(summary_with_npi)


In [ ]:
write.fst(summary_with_npi,"summary_with_npi_before_join.fst")

## Physician integration status

Here, I will find which physicians are integrated.


### A method of finding codes that are not exclusive to hospitals or non-hospital places
We can exclude these HCPCS codes and only include codes that are not exclusive to hospitals.

In [10]:
exclusive_hospital_code_finder = function (data,
                                           threshold=0.05,
                                           integrated_place_of_service_codes = c("19", "22"),
                                           all_place_of_service_codes = c("11", "19", "22")){
  require(dtplyr)
  require(tidyverse)
  
  result = data %>%
  filter(LINE_PLACE_OF_SRVC_CD %in% all_place_of_service_codes)%>%
  group_by(HCPCS_CD) %>%
  summarise(prp_in_facility = nrow(.[LINE_PLACE_OF_SRVC_CD %in% integrated_place_of_service_codes])/n()
           )%>%
  as.data.table
  
  exclusive_codes = result[prp_in_facility<threshold | prp_in_facility>(1-threshold),HCPCS_CD]
  

  return(exclusive_codes)
  
}

exclusive_hospital_codes=exclusive_hospital_code_finder(carrier_data_all_years)

### A function to find integrated docs

In [12]:
#calculate and add physician integration data
#this only uses visits to see if a physician is integrated or not (codde list)

physician_integration_finder = function(data,
                                        integrated_place_of_service_codes = c("19", "22"),
                                        all_place_of_service_codes = c("11", "19", "22"),
                                        #integration_threshold = 0.5,
                                        office_code_list = c(
                                          "99201",
                                          "99202",
                                          "99203",
                                          "99204",
                                          "99205",
                                          "99211",
                                          "99212",
                                          "99213",
                                          "99214",
                                          "99215"
                                        ),
                                       exclusive_hospital_codes) {
  require(dtplyr)
  require(tidyverse)
  
  #data = subset(data, HCPCS_CD %in% code_list)
  result = data %>%
  mutate(
    is_facility = LINE_PLACE_OF_SRVC_CD %in% integrated_place_of_service_codes,
    is_all = LINE_PLACE_OF_SRVC_CD %in% all_place_of_service_codes,
    is_office_visit = HCPCS_CD %in% office_code_list,
    has_non_exclusive_code = HCPCS_CD %!in% exclusive_hospital_codes
  ) %>%
  group_by(PRF_PHYSN_NPI, year) %>%
  summarise(
    in_facility_visits_count = sum(is_facility*is_office_visit, na.rm = T),
    in_all_visits_count = sum(is_all*is_office_visit, na.rm = T),
    in_facility_non_exclusive_HCPCS_count = sum(is_facility*has_non_exclusive_code, na.rm = T),
    in_all_non_exclusive_HCPCS_count = sum(is_all*has_non_exclusive_code, na.rm = T),
    in_facility_count = sum(is_facility, na.rm = T),
    in_all_count = sum(is_all, na.rm = T)
  ) %>%
  mutate(
    in_facility_visits_prp = in_facility_visits_count / in_all_visits_count,
    in_facility_non_exclusive_HCPCS_prp = in_facility_non_exclusive_HCPCS_count / in_all_non_exclusive_HCPCS_count,    
    in_facility_prp = in_facility_count / in_all_count
  )%>%
  as.data.table()
}

physician_integration_stats = physician_integration_finder(carrier_data_all_years,exclusive_hospital_codes=exclusive_hospital_codes)

`summarise()` has grouped output by 'PRF_PHYSN_NPI'. You can override using the `.groups` argument.


In [13]:
tail(physician_integration_stats)

PRF_PHYSN_NPI,year,in_facility_visits_count,in_all_visits_count,in_facility_non_exclusive_HCPCS_count,in_all_non_exclusive_HCPCS_count,in_facility_count,in_all_count,in_facility_visits_prp,in_facility_non_exclusive_HCPCS_prp,in_facility_prp
<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1992997522,2013,2,2,10,10,10,10,1,1,1
1992998157,2013,0,0,0,0,0,0,NaN,NaN,NaN
1992998207,2013,0,0,0,0,0,5,NaN,NaN,0
1992999122,2013,0,8,0,8,0,8,0,0,0
9999999991,2013,0,1,0,2,0,2,0,0,0
9999999992,2013,0,3,0,16,0,71,0,0,0


### Add integration status of physicians
This function will add the integration status of most common physicians to each patient's summary data.

In [ ]:
#summary_with_npi=read.fst("./summary_with_npi_before_join.fst", as.data.table = T)

physician_integration_stats=read.fst("./physician_integration_stats_before_join.fst", as.data.table = T)

In [ ]:
#rename columns
rename_last = function(data, how_many, new_names) {
  total_cols = ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}
add_integration_status=function(data, physician_integration_stats){
  
  data_selected=data[,c("DESY_SORT_KEY",
               "year",
               "most_common_physician_PRF_PHYSN_NPI",
               "most_common_primary_care_physician_PRF_PHYSN_NPI"
              )]
  
  most_common_physician = left_join(
    data_selected,
    physician_integration_stats,
    by = c(
      "most_common_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI", "year" = "year")
  ) %>% as.data.table()
  
  most_common_physician=most_common_physician[,-c("most_common_primary_care_physician_PRF_PHYSN_NPI")]
  
  rename_last(
    most_common_physician,
    ncol(physician_integration_stats)-2,
    paste("most_common_physician_",colnames(physician_integration_stats)[3:ncol(physician_integration_stats)],sep="")
    )
    
    
  most_common_primary_care = left_join(
    data_selected,
    physician_integration_stats,
    by = c(
      "most_common_primary_care_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI", "year" = "year")
  ) %>% as.data.table()
  
  most_common_primary_care=most_common_primary_care[,-c("most_common_physician_PRF_PHYSN_NPI")]  

  rename_last(
    most_common_primary_care,
    ncol(physician_integration_stats)-2,
    paste("most_common_primary_care_physician_",colnames(physician_integration_stats)[3:ncol(physician_integration_stats)],sep="")
    )
  
  physician_data=
  full_join(most_common_physician[,-("most_common_physician_PRF_PHYSN_NPI")],
            most_common_primary_care[,-("most_common_primary_care_physician_PRF_PHYSN_NPI")],
            by=c("DESY_SORT_KEY","year")
           )
  result=full_join(data,
                   physician_data,
                   by=c("DESY_SORT_KEY","year")
           )%>%
  as.data.table
  
  
  return(result)
}

summary_with_physician_integration_stats=add_integration_status(data = summary_with_npi,
                                                                physician_integration_stats = physician_integration_stats)






In [ ]:
head(summary_with_physician_integration_stats)

In [ ]:
write.fst(summary_with_physician_integration_stats,"summary_with_physician_integration_stats.fst")

# Comparisons and analyses

## Reading the patient and physician integration results

In [5]:
yearly_calculations =
read_fst("calculation_results.fst"
         ,as.data.table = T) 

## Adding metropolitan status
I will add the metropolitan statuses of the patient counties using the USDA data

In [6]:
rural_urban_data = readxl::read_xls("physician_data/ruralurbancodes2013.xls")%>%as.data.table()
head(rural_urban_data)
cross_walk_rural_urban=read.csv(file = "physician_data/xwalk2018.csv")%>%as.data.table()
head(cross_walk_rural_urban)

FIPS,State,County_Name,Population_2010,RUCC_2013,Description
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
01001,AL,Autauga County,54571,2,"Metro - Counties in metro areas of 250,000 to 1 million population"
01003,AL,Baldwin County,182265,3,"Metro - Counties in metro areas of fewer than 250,000 population"
01005,AL,Barbour County,27457,6,"Nonmetro - Urban population of 2,500 to 19,999, adjacent to a metro area"
01007,AL,Bibb County,22915,1,Metro - Counties in metro areas of 1 million population or more
01009,AL,Blount County,57322,1,Metro - Counties in metro areas of 1 million population or more
01011,AL,Bullock County,10914,6,"Nonmetro - Urban population of 2,500 to 19,999, adjacent to a metro area"


County.Name,State,SSACD,FIPS.County.Code,CBSA,CBSA.Name
<chr>,<chr>,<int>,<int>,<int>,<chr>
AUTAUGA,AL,1000,1001,33860,"Montgomery, AL"
BALDWIN,AL,1010,1003,19300,"Daphne-Fairhope-Foley, AL"
BARBOUR,AL,1020,1005,NA,
BIBB,AL,1030,1007,13820,"Birmingham-Hoover, AL"
BLOUNT,AL,1040,1009,13820,"Birmingham-Hoover, AL"
BULLOCK,AL,1050,1011,NA,


In [7]:
yearly_calculations[,SSACD:=as.integer(paste(STATE_CODE,COUNTY_CODE,sep=""))]
yearly_calculations=left_join(yearly_calculations,cross_walk_rural_urban,by="SSACD")%>%
mutate(FIPS.County.Code=as.character(FIPS.County.Code))%>%
left_join(.,rural_urban_data,by=c("FIPS.County.Code"="FIPS"))%>%
as.data.table()
yearly_calculations[,c("SSACD","County.Name","State.x","FIPS.County.Code","CBSA","CBSA.Name","State.y","County_Name","Population_2010","Description"):=NULL]
head(yearly_calculations)


#STATE_CODE	COUNTY_CODE	SEX_CODE	RACE_CODE	

Warning message in eval(jsub, SDenv, parent.frame()):
“NAs introduced by coercion”


DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
100000015,2013,7,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,13,22,160,1,1,76,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1285600932,2,11,1285600932,2,11,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,1
100000015,2014,10,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,14,22,160,1,1,77,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,1
100000015,2015,6,2,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,15,22,160,1,1,78,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,1
100000015,2016,9,2,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2016,22,160,1,1,79,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1730170630,4,06,1730170630,4,11,0,21,0,21,4,41,0.00000000,0.0000000,0.09756098,0,21,0,21,4,41,0.00000000,0.0000000,0.09756098,1
100000015,2017,14,3,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2017,22,160,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1043207350,3,11,1043207350,3,11,58,58,68,68,69,69,1.00000000,1.0000000,1.00000000,58,58,68,68,69,69,1.00000000,1.0000000,1.00000000,1
100000015,2018,16,2,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2018,22,160,1,1,81,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,10432

## Adding sex, race, and state names from codes

In [9]:
add_personal_details=function(data){
    
  require(tidyverse)
  require(dtplyr)
  require(lubridate)
  
  result=data %>%
  #left_join(.,census_and_state_codes[,-1],by=c("STATE_CODE"="state_code"))%>%
  left_join(.,race_codes,by=c("RACE_CODE"="race_code"))%>%
  left_join(.,sex_codes,by=c("SEX_CODE"="sex_code"))%>%
  mutate(
    age_group=case_when(
      AGE<75 & AGE>64 ~ "65-74",
      AGE>74 & AGE<85 ~ "75-84",
      AGE>84 ~ "85+"
    ),
    urban=(RUCC_2013<=3)
  )%>%
  as.data.table()
  
  return(result)
}

#STATE_CODE	COUNTY_CODE	SEX_CODE	RACE_CODE	

In [10]:
yearly_calculations=add_personal_details(yearly_calculations)

Loading required package: lubridate


Attaching package: ‘lubridate’


The following object is masked from ‘package:reshape’:

    stamp


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week, yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




## Adding HMO indicator and buy in sums

In [37]:
yearly_calculations[,HMO_INDICATOR_sum:=sum(
  HMO_INDICATOR01 == 0 ,
  HMO_INDICATOR02 == 0 ,
  HMO_INDICATOR03 == 0 ,
  HMO_INDICATOR04 == 0 ,
  HMO_INDICATOR05 == 0 ,
  HMO_INDICATOR06 == 0 ,
  HMO_INDICATOR07 == 0 ,
  HMO_INDICATOR08 == 0 ,
  HMO_INDICATOR09 == 0 ,
  HMO_INDICATOR10 == 0 ,
  HMO_INDICATOR11 == 0 ,
  HMO_INDICATOR12 == 0 ,
  na.rm=T
),by=1:nrow(yearly_calculations)]

yearly_calculations[,ENTITLEMENT_BUY_IN_IND_sum:=sum(
  ENTITLEMENT_BUY_IN_IND01 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND02 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND03 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND04 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND05 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND06 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND07 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND08 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND09 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND10 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND11 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND12 %in% c("3", "C"),
  na.rm=T
),by=1:nrow(yearly_calculations)]

## Adding months alive within the year of death

In [47]:
yearly_calculations[,months_alive:=as.numeric(substr(DATE_OF_DEATH,5,6))]

## Filtering the data for analysis

In [52]:
length(yearly_calculations$DESY_SORT_KEY)

[1] 24324809

In [53]:
data_for_modelling_filter=function(data){
  
  library(tidyverse)
  library(dtplyr)
  
  data%>%
  filter(STATE_CODE %!in% non_us_state_codes &
         
         AGE>=65&
         
         ((!is.na(months_alive) & HMO_INDICATOR_sum>=months_alive) |
          (is.na(months_alive) & HMO_INDICATOR_sum==12))&
          
          ((!is.na(months_alive) & ENTITLEMENT_BUY_IN_IND_sum>=months_alive) |
          (is.na(months_alive) & ENTITLEMENT_BUY_IN_IND_sum==12))
          
        )%>%
  as.data.table()
}



In [54]:
yearly_calculations=data_for_modelling_filter(yearly_calculations)

In [55]:
length(yearly_calculations$DESY_SORT_KEY)

[1] 18137669

In [62]:
head(yearly_calculations[ENTITLEMENT_BUY_IN_IND_sum<12])

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>
100000315,2013,3,1,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,10,510,2,1,70,1,0,C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20130103,20130103,V,2013,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,White,Female,65-74,TRUE,12,1,1
100000437,2014,4,0,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,14,30,080,1,1,83,0,0,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20140701,20140701,V,2014,1902988124,2,27,NA,NA,NA,0,18,0,10,0,18,0,0,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,1,White,Male,75-84,TRUE,12,7,7
100000563,2014,25,4,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,14,10,040,2,2,93,0,0,C,C,C,C,C,C,C,C,C,C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20141003,20141003,V,2014,1598995185,4,08,1598995185,4,08,0,51,0,109,0,190,0,0,0,0,51,0,109,0,190,0,0,0,2,Black,Female,85+,TRUE,12,10,10
100001177,2013,3,1,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,01,260,1,1,81,1,0,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20130825,20130825,V,2013,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,White,Male,75-84,NA,12,8,8
100001395,2016,24,5,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,2016,36,440,2,1,79,0,0,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20160721,20160721,V,2016,1316907827,3,08,1316907827,3,08,0,44,0,217,0,380,0,0,0,0,44,0,217,0,380,0,0,0,1,White,Female,75-84,TRUE,12,7,7
100001851,2013,11,0,TRUE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,13,33,590,1,5,65,1,0,3,3,3,3,0,0,0,0

# Comparison functions